In [94]:
import json
import requests

create_index_url: str = "http://0.0.0.0:8000/create_index"
add_documents_url: str = "http://0.0.0.0:8000/add_documents"
get_document_url: str = "http://0.0.0.0:8000/get_document"


courses_path: str = (
    "/Users/egor/Documents/code/UrTraining/ml/scripts/selected_courses_with_ids.json"
)

courses = json.load(open(courses_path))
courses_to_upload = []
for course in courses:
    # Stringify course
    course_parts = []
    for key, value in course.items():
        if isinstance(value, list):
            value_str = ", ".join(str(v) for v in value)
        else:
            value_str = str(value)
        course_parts.append(f"{key}: {value_str}")
    course_str = "\n\n".join(course_parts)
    courses_to_upload.append({
        "id": course.get("id"),
        "content": course_str
    })

### Create Index

In [95]:
res = requests.post(create_index_url, json={
    "name": "index_v3",
    "dimension": 1024,
    "distance_metric": "L2",
    "index_type": "Flat",
})

res.json()

{'success': True,
 'message': "Index 'index_v3' created successfully",
 'index_name': 'index_v3'}

In [ ]:
from itertools import batched

# Split documents into 4 batches
batch_size = 2

url: str = "http://0.0.0.0:8000/add_documents"

results = []
for i, batch_documents in enumerate(batched(courses_to_upload, batch_size)):
    res = requests.post(
        url,
        json={
            "index_name": "index_v3",
            "documents": list(batch_documents),
        },
    )
    results.append(res.json())
    import time
    time.sleep(2)
    print(f"Batch {i+1} response: {res.status_code}")

In [49]:
ids = [result["document_ids"] for result in results]
ids_flat = [item for sublist in ids for item in sublist]

In [54]:
selected_courses = [courses[i] for i in selected_indexes]

In [60]:
selected_courses_with_ids = [
    {**course, "id": doc_id} for course, doc_id in zip(selected_courses, ids_flat)
]

In [63]:
json.dump(selected_courses_with_ids, open("selected_courses_with_ids.json", "w"), indent=4)